In [1]:
import yfinance as yf
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import itertools
from collections import defaultdict

def get_exchange_rates():
    currencies = ['USD', 'EUR', 'JPY', 'GBP', 'CHF', 'CAD', 'AUD', 'NZD', 'CNY', 'INR']
    rates = {}
    for base in currencies:
        for quote in currencies:
            if base != quote:
                pair = f"{base}{quote}=X"
                try:
                    data = yf.Ticker(pair).history(period="max")
                    if not data.empty:
                        rates[f"{base}/{quote}"] = data['Close'].iloc[-1]
                    else:
                        print(f"No data available for {pair}")
                except Exception as e:
                    print(f"Error fetching data for {pair}: {e}")
    return rates

def predict_arima(base, quote):
    pair = f"{base}/{quote}"
    try:
        data = yf.Ticker(f"{base}{quote}=X").history(period="max")
        if data.empty or len(data) < 50:
            return None
        
        # Ensure the data has a proper date index with frequency
        data.index = pd.to_datetime(data.index)
        data = data.asfreq('D')  # Set frequency to daily if not already
        
        train_data, test_data = train_test_split(data['Close'], test_size=0.2, shuffle=False)
        
        # Train ARIMA model
        model = ARIMA(train_data, order=(1, 1, 1))
        model_fit = model.fit()
        
        # Forecast next 3 days
        forecast = model_fit.forecast(steps=3)
        return forecast[-1]  # Return prediction for the next day
    except Exception as e:
        print(f"Error predicting {base}/{quote} using ARIMA: {str(e)}")
        return None

def main():
    print("Fetching exchange rates...")
    rates = get_exchange_rates()
    
    if not rates:
        print("No exchange rate data available. Exiting.")
        return
    
    print("Predicting exchange rates using ARIMA...")
    predicted_rates = {}
    currencies = ['USD', 'EUR', 'JPY', 'GBP', 'CHF', 'CAD', 'AUD', 'NZD', 'CNY', 'INR']
    
    for base in currencies:
        for quote in currencies:
            if base != quote:
                prediction = predict_arima(base, quote)
                if prediction is not None:
                    predicted_rates[f"{base}/{quote}"] = prediction
                else:
                    print(f"Failed to predict {base}/{quote}")
    
    print("Predicted rates:")
    print(predicted_rates)

if __name__ == "__main__":
    main()

Fetching exchange rates...
Predicting exchange rates using ARIMA...


/var/folders/56/b65znxrd40327qx21xzdtl5c0000gn/T/ipykernel_26434/2893991794.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return forecast[-1]  # Return prediction for the next day
/var/folders/56/b65znxrd40327qx21xzdtl5c0000gn/T/ipykernel_26434/2893991794.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return forecast[-1]  # Return prediction for the next day
/var/folders/56/b65znxrd40327qx21xzdtl5c0000gn/T/ipykernel_26434/2893991794.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFra

Failed to predict CNY/INR


/opt/anaconda3/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/var/folders/56/b65znxrd40327qx21xzdtl5c0000gn/T/ipykernel_26434/2893991794.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return forecast[-1]  # Return prediction for the next day
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/var/folders/56/b65znxrd40327qx21xzdtl5c0000gn/T/ipykernel_26434/2893991794.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future ve

Predicted rates:
{'USD/EUR': 0.9247445774952272, 'USD/JPY': 113.40045839313073, 'USD/GBP': 0.8274576071805474, 'USD/CHF': 0.9673681697795717, 'USD/CAD': 1.3947487189263903, 'USD/AUD': 1.3799970659680243, 'USD/NZD': 1.683845600472946, 'USD/CNY': 7.0292596880185245, 'USD/INR': 75.50418376193731, 'EUR/USD': 1.0815116686314328, 'EUR/JPY': 117.09758719390881, 'EUR/GBP': 0.8833211058112547, 'EUR/CHF': 1.0568404991259783, 'EUR/CAD': 1.523987487270199, 'EUR/AUD': 1.68297998359961, 'EUR/NZD': 1.8203461258011893, 'EUR/CNY': 7.660379036824901, 'EUR/INR': 81.51002057591087, 'JPY/USD': 0.008818769658052726, 'JPY/EUR': 0.008220181660225919, 'JPY/GBP': 0.010995986116428534, 'JPY/CHF': 0.009049240380361381, 'JPY/CAD': 0.012339232677848375, 'JPY/AUD': 0.014488202312938523, 'JPY/NZD': 0.014320877390978128, 'JPY/CNY': 0.06604397602880997, 'JPY/INR': 0.7027370572766773, 'GBP/USD': 1.2089251129017016, 'GBP/EUR': 1.149498266603761, 'GBP/JPY': 129.60082148176377, 'GBP/CHF': 1.1986934573946093, 'GBP/CAD': 1.7

/var/folders/56/b65znxrd40327qx21xzdtl5c0000gn/T/ipykernel_26434/2893991794.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return forecast[-1]  # Return prediction for the next day
